In [2]:
import random
from http import HTTPStatus
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

## 单轮对话

In [46]:
# initialize
model="qwen-turbo"
# defination
def call_with_messages(system_message, user_message):
    messages = [{'role': 'system', 'content':system_message},
                {'role': 'user', 'content': user_message}]
    response = Generation.call(model=model,
                               messages=messages,
                               # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                               #seed=random.randint(1, 10000),
                               # 将输出设置为"message"格式
                               result_format="message")
    if response.status_code == HTTPStatus.OK:
        return response,response.output.choices[0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))

In [51]:
response,content=call_with_messages('你是一名专业的机器学习工程师。','解释EM算法的大致思路？')

In [52]:
print(response,'\n-------------\n',content)

{"status_code": 200, "request_id": "0380bc65-e5d4-9e56-8e0e-4b695fc5883f", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "EM（Expectation-Maximization）算法是一种迭代的无监督参数估计方法，主要用于高斯混合模型（Gaussian Mixture Model, GMM）等概率模型中。EM算法的主要思路分为两个步骤：\n\n1. **E步（Expectation Step, 预期步）**：\n   在这个步骤中，给定当前模型参数（如高斯混合模型中的每个成分的均值、方差和权重），我们计算观察数据属于每个潜在类别的概率。这被称为“期望”因为我们计算的是在给定当前参数下的观测数据属于各个类别的条件概率。具体来说，对于每个观测样本，我们计算它属于每个混合成分的概率，然后更新每个样本的“隐藏状态”，即它最可能属于哪个类别。\n\n2. **M步（Maximization Step, 最大化步）**：\n   在这个步骤中，我们基于E步的结果来最大化似然函数或数据的对数似然。由于似然函数通常包含一个关于隐藏变量的期望，它不能直接求导，所以我们对期望进行处理，将其转换为只依赖于可见数据的函数。然后，我们找到使这个新函数最大的参数值，即更新混合模型的参数，比如每个高斯分布的均值、方差和权重。\n\n这两个步骤交替进行，直到模型参数不再显著改变或者达到预设的迭代次数。EM算法保证了在每次迭代后，模型的似然函数不会下降（局部最优）。虽然它不保证全局最优，但在许多情况下，它能提供很好的结果。"}}]}, "usage": {"input_tokens": 27, "output_tokens": 327, "total_tokens": 354}} 
-------------
 EM（Expectation-Maximization）算法是一种迭代的无监督参数估计方法，主要用于高斯混合模型（Gaussian Mixture Mo

## 多轮对话

In [2]:
from http import HTTPStatus
from dashscope import Generation


def multi_round():
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': '如何做西红柿炖牛腩？'}]
    response = Generation.call(model="qwen-turbo",
                               messages=messages,
                               # 将输出设置为"message"格式
                               result_format='message')
    if response.status_code == HTTPStatus.OK:
        print(response)
        # 将assistant的回复添加到messages列表中
        messages.append({'role': response.output.choices[0]['message']['role'],
                         'content': response.output.choices[0]['message']['content']})
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        # 如果响应失败，将最后一条user message从messages列表里删除，确保user/assistant消息交替出现
        messages = messages[:-1]
    # 将新一轮的user问题添加到messages列表中
    messages.append({'role': 'user', 'content': '不放糖可以吗？'})
    # 进行第二轮模型的响应
    response = Generation.call(model="qwen-turbo",
                               messages=messages,
                               result_format='message',  # 将输出设置为"message"格式
                               )
    if response.status_code == HTTPStatus.OK:
        print(response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))


if __name__ == '__main__':
    multi_round()

{"status_code": 200, "request_id": "360d972f-f4d3-9e91-ad83-0ab982c88c0d", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "材料：\n牛腩500克，西红柿3个，大葱1根，生姜2片，八角2颗，香叶2片，桂皮1小块，干辣椒3个，料酒2汤匙，生抽2汤匙，老抽1汤匙，冰糖30克，盐适量，清水适量\n\n步骤：\n\n1. 牛腩切块，用清水浸泡半小时，去除血水和杂质，然后沥干备用。\n\n2. 热锅凉油，放入冰糖小火慢慢熬化，糖色炒至红色时加入牛肉块，翻煎至表面微焦，这样可以让牛肉更加香甜。\n\n3. 加入切好的葱段、姜片、八角、香叶、桂皮和干辣椒，继续翻煎出香味。\n\n4. 倒入料酒，翻煮一会儿让酒精挥发，去腥增香。\n\n5. 加入生抽、老抽，翻炒均匀使牛肉上色。\n\n6. 倒入足够的热水，水量要没过牛肉，大火烧开后撇去浮沫。\n\n7. 西红柿切块，等牛肉烧沸后，转中小火，加入西红柿块，盖上锅盖慢慢炖煮，炖煮的过程中可以适当调整火力，保持锅内汤汁微微沸腾。\n\n8. 炖煮约1-1.5小时，直到牛肉变得软烂，汤汁浓郁。期间如果水分蒸发过多，可以适量加水。\n\n9. 最后根据个人口味加盐调味，再炖煮一会儿让味道充分融合即可。\n\n10. 出锅前可以尝一下味道，如需增色或者提鲜，可以撒上一些葱花或者香菜。\n\n西红柿炖牛腩就做好了，这道菜色泽红亮，肉质酥烂，酸甜可口，非常适合秋冬季节食用。"}}]}, "usage": {"input_tokens": 26, "output_tokens": 397, "total_tokens": 423}}
{"status_code": 200, "request_id": "312ee357-3ecc-921c-97e2-ea3a37c70560", "code": "", "message": "", "output": {"text": null, "finish_reason"

In [21]:
def multi_round():
    answer=[]
    # 第一轮模型的响应
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': '如何做西红柿炖牛腩？'}]
    response = Generation.call(model="qwen-turbo",
                               messages=messages,
                               temperature=1,
                               # 将输出设置为"message"格式
                               result_format='message')
    if response.status_code == HTTPStatus.OK:
        # print(response)
        answer.append(response.output.choices[0]['message']['content'])
        # 将assistant的回复添加到messages列表中
        messages.append({'role': response.output.choices[0]['message']['role'],'content': response.output.choices[0]['message']['content']})
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        # 如果响应失败，将最后一条user message从messages列表里删除，确保user/assistant消息交替出现
        messages = messages[:-1]
    # 将新一轮的user问题添加到messages列表中
    messages.append({'role': 'user', 'content': '不放糖可以吗？'})


    # 进行第二轮模型的响应
    response = Generation.call(model="qwen-turbo",
                               messages=messages,
                               temperature=1,
                               result_format='message',  # 将输出设置为"message"格式
                               )
    if response.status_code == HTTPStatus.OK:
        # print(response)
        answer.append(response.output.choices[0]['message']['content'])
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    
    return answer

In [22]:
answer=multi_round()
for i in range(len(answer)):
    print(answer[i])
    print('---------------------------')

西红柿炖牛腩是一道美味的家常菜，下面是简单的步骤：

所需材料：
- 牛腩500克
- 西红柿4个
- 洋葱1个
- 大蒜3瓣
- 生姜2片
- 料酒适量
- 酱油适量
- 白糖适量
- 八角2颗
- 桂皮1小块
- 干辣椒2-3个（可选）
- 清水适量
- 橄榄油或者食用油适量

步骤：
1. 牛腩清洗干净，切成大块，放入开水中焯水去血沫，捞出备用。这个步骤可以去掉牛肉的腥味。
2. 西红柿洗净切块，洋葱切丁，大蒜和生姜切片。
3. 热锅凉油，加入洋葱、大蒜和姜片炒香。
4. 加入焯过水的牛腩块，翻煎至两面微黄，让牛腩表面形成焦香。
5. 倒入料酒，煮一会儿以去腥味。
6. 加入切好的西红柿块，继续翻炒均匀，让西红柿汁液出来。
7. 加入酱油、白糖、八角、桂皮和干辣椒（如果喜欢辣的话），调味料根据个人口味调整。
8. 倒入足够的清水，水量要没过牛腩，大火烧开后转小火慢慢炖煮。盖上锅盖，让牛腩充分吸收番茄的酸甜和调料的味道。
9. 炖煮约1-1.5小时，直到牛腩变得软烂，汤汁浓稠即可。期间可以适当尝味道，根据需要再加盐或酱油。
10. 最后撒上一些葱花或者香菜点缀，一道美味的西红柿炖牛腩就做好了。

这道菜的关键在于慢炖，让牛腩充分软烂，同时吸收西红柿的酸甜味道。享受烹饪过程的同时，也可以根据自己的喜好调整食材和调料。
---------------------------
当然可以。西红柿炖牛腩的甜度主要是由西红柿自然的糖分提供的，如果你不喜欢甜味，可以省略或减少白糖。在炖煮过程中，西红柿会慢慢释放出天然的酸甜口感，这足以提升整道菜的风味。如果你更倾向于原汁原味，那么不放糖也是很好的选择。根据个人口味来调整就是了。
---------------------------
